# Regression

In [ ]:
using Pkg
Pkg.activate(".")

In [ ]:
using Plots
using Statistics
using StatsBase
using PyCall
using DataFrames
using GLM
using Tables
using XLSX
using MLBase
using RDatasets
using LsqFit

In [ ]:
xvals = repeat(1:0.5:10, inner=2)
yvals = 3 .+ xvals .+ 2 .* rand(length(xvals)) .-1

In [ ]:
scatter(xvals, yvals, color=:black, leg=false)

In [ ]:
function find_best_fit(xvals, yvals)
    meanx = mean(xvals)
    meany = mean(yvals)
    stdx = std(xvals)
    stdy = std(yvals)
    r = cor(xvals, yvals)
    a = r*stdy/stdx
    b = meany - a*meanx 
    return a, b
end

In [ ]:
a, b = find_best_fit(xvals, yvals)
ynew = a .* xvals .+ b

In [ ]:
np = pyimport("numpy");

In [ ]:
xdata = xvals;
ydata = yvals;
@time myfit = np.polyfit(xdata, ydata, 1);
ynew2 = collect(xdata) .* myfit[1] .+ myfit[2];
scatter(xvals, yvals)
plot!(xvals, ynew)